In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
#from google.colab import drive
#import cupy as cp
#import cudf
import dask
#from dask_ml.preprocessing import Categorizer
#import dask.dataframe as dd
import pickle  # To save mappings for decoding
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
#import dask.array as da
from xgboost.dask import DaskXGBRegressor
import time 



In [2]:
import dask.dataframe as dd
import dask.array as da
from sklearn.linear_model import SGDRegressor  # For batch-by-batch linear regression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from dask_ml.model_selection import train_test_split


In [26]:

data = dd.read_parquet("C:/Users/Himanshu/Downloads/proc_data_updated.parquet",blocksize="64MB")


In [27]:
# Define the dtypes dictionary for all columns
dtypes = {
    'DepTime': 'float16',
    'CRSDepTime': 'float16',
    'ArrTime': 'float16',
    'CRSArrTime': 'float16',
    'ArrDelay': 'float16',
    'DepDelay': 'float16',
    'TaxiIn': 'float16',
    'TaxiOut': 'float16',
    'CarrierDelay': 'float16',
    'WeatherDelay': 'float16',
    'NASDelay': 'float16',
    'SecurityDelay': 'float16',
    'LateAircraftDelay': 'float16'
}
features = ['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime',
            'ArrDelay', 'TaxiIn', 'TaxiOut', 'CarrierDelay', 'WeatherDelay',
            'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
target = 'DepDelay'
# Apply dtypes to the Dask DataFrame
data = data.astype(dtypes)

In [28]:
data.head(5)

,DepTime,CRSDepTime,ArrTime,CRSArrTime,ArrDelay,DepDelay,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,623.0,630.0,901.0,915.0,-14.0,-7.0,7.0,11.0,0.0,0.0,0.0,0.0,0.0
1,621.0,630.0,911.0,915.0,-4.0,-9.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0
2,633.0,630.0,920.0,915.0,5.0,3.0,4.0,15.0,0.0,0.0,0.0,0.0,0.0
3,627.0,630.0,859.0,915.0,-16.0,-3.0,4.0,10.0,0.0,0.0,0.0,0.0,0.0
4,635.0,630.0,918.0,915.0,3.0,5.0,3.0,13.0,0.0,0.0,0.0,0.0,0.0


In [29]:
# Define features and target
features = ['DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime',
            'ArrDelay', 'TaxiIn', 'TaxiOut', 'CarrierDelay', 'WeatherDelay',
            'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
target = 'DepDelay'

In [30]:
X = data[features].to_dask_array(lengths=True)
y = data[target].to_dask_array(lengths=True)

In [31]:
# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


#n_partitions = 300, n_worker = 2, threads_per_worker = 2

In [27]:
import time
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask.diagnostics import ProgressBar

# XGBoost related imports
from xgboost.dask import DaskXGBRegressor

# Metrics for model evaluation
from sklearn.metrics import mean_squared_error, r2_score

# Initialize Dask Client
client = Client(n_workers=2, threads_per_worker=2) # You can specify a scheduler address or leave it for local cluster

# Your dataset needs to be loaded as Dask DataFrames (assuming X_train, y_train, X_test, y_test are Dask DataFrames)
# Example:
# X_train = dd.from_pandas(pandas_X_train, npartitions=4)
# y_train = dd.from_pandas(pandas_y_train, npartitions=4)
# X_test = dd.from_pandas(pandas_X_test, npartitions=4)
# y_test = dd.from_pandas(pandas_y_test, npartitions=4)

# Initialize the Dask-XGBoost Regressor
xgb_model = DaskXGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3)

# Train the Model
print("Training the model...")
start_time = time.time()  # Start timer

with ProgressBar():
    xgb_model.fit(X_train, y_train)
end_time = time.time()  # End timer

# Calculate training time
training_time = end_time - start_time
print(f"Training completed in {training_time:.2f} seconds.")

# Make Predictions
print("Predicting...")
y_pred = xgb_model.predict(X_test)

# Evaluate the Model
# y_test and y_pred are already NumPy arrays, no need to call .compute()
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


# Shut down the client
client.close()


c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55715 instead
  warnings.warn(
Windows is not officially supported for dask/xgboost, contribution are welcomed.
Windows is not officially supported for dask/xgboost, contribution are welcomed.


Training the model...


c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.61 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 269.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Windows is not officially supported for dask/xgboost, contribution are welcomed.


Training completed in 275.81 seconds.
Predicting...


Windows is not officially supported for dask/xgboost, contribution are welcomed.
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.63 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.63 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1198: RuntimeWarning: overflow encountered in square
  weight * (y_true - np.

Mean Squared Error: 64.46044158935547
R-squared: 1.0


n_partitions = 300 with n_worker = 4, threads_per_worker = 2

In [25]:
import time
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask.diagnostics import ProgressBar

# XGBoost related imports
from xgboost.dask import DaskXGBRegressor

# Metrics for model evaluation
from sklearn.metrics import mean_squared_error, r2_score

# Initialize Dask Client
client = Client(n_workers=4, threads_per_worker=2) # You can specify a scheduler address or leave it for local cluster

# Your dataset needs to be loaded as Dask DataFrames (assuming X_train, y_train, X_test, y_test are Dask DataFrames)
# Example:
# X_train = dd.from_pandas(pandas_X_train, npartitions=4)
# y_train = dd.from_pandas(pandas_y_train, npartitions=4)
# X_test = dd.from_pandas(pandas_X_test, npartitions=4)
# y_test = dd.from_pandas(pandas_y_test, npartitions=4)

# Initialize the Dask-XGBoost Regressor
xgb_model = DaskXGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3)

# Train the Model
print("Training the model...")
start_time = time.time()  # Start timer

with ProgressBar():
    xgb_model.fit(X_train, y_train)
end_time = time.time()  # End timer

# Calculate training time
training_time = end_time - start_time
print(f"Training completed in {training_time:.2f} seconds.")

# Make Predictions
print("Predicting...")
y_pred = xgb_model.predict(X_test)

# Evaluate the Model
# y_test and y_pred are already NumPy arrays, no need to call .compute()
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


# Shut down the client
client.close()


c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55415 instead
  warnings.warn(
Windows is not officially supported for dask/xgboost, contribution are welcomed.
Windows is not officially supported for dask/xgboost, contribution are welcomed.


Training the model...


c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.61 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 269.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Windows is not officially supported for dask/xgboost, contribution are welcomed.


Training completed in 188.69 seconds.
Predicting...


Windows is not officially supported for dask/xgboost, contribution are welcomed.
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.63 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.63 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1198: RuntimeWarning: overflow encountered in square
  weight * (y_true - np.

Mean Squared Error: 64.46044158935547
R-squared: 1.0


# n_partitions = 300

In [ ]:
import time
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask.diagnostics import ProgressBar

# XGBoost related imports
from xgboost.dask import DaskXGBRegressor

# Metrics for model evaluation
from sklearn.metrics import mean_squared_error, r2_score

# Initialize Dask Client
client = Client()  # You can specify a scheduler address or leave it for local cluster

# Your dataset needs to be loaded as Dask DataFrames (assuming X_train, y_train, X_test, y_test are Dask DataFrames)
# Example:
# X_train = dd.from_pandas(pandas_X_train, npartitions=4)
# y_train = dd.from_pandas(pandas_y_train, npartitions=4)
# X_test = dd.from_pandas(pandas_X_test, npartitions=4)
# y_test = dd.from_pandas(pandas_y_test, npartitions=4)

# Initialize the Dask-XGBoost Regressor
xgb_model = DaskXGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3)

# Train the Model
print("Training the model...")
start_time = time.time()  # Start timer

with ProgressBar():
    xgb_model.fit(X_train, y_train)
end_time = time.time()  # End timer

# Calculate training time
training_time = end_time - start_time
print(f"Training completed in {training_time:.2f} seconds.")

# Make Predictions
print("Predicting...")
y_pred = xgb_model.predict(X_test)

# Evaluate the Model
y_test = y_test.compute()  # Bring to memory
y_pred = y_pred.compute()  # Bring to memory

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Shut down the client
client.close()


Windows is not officially supported for dask/xgboost, contribution are welcomed.
Windows is not officially supported for dask/xgboost, contribution are welcomed.


Training the model...


c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.61 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 269.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Windows is not officially supported for dask/xgboost, contribution are welcomed.


Training completed in 163.68 seconds.
Predicting...


Windows is not officially supported for dask/xgboost, contribution are welcomed.
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 269.70 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.63 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1198: RuntimeWarning: overflow encountered in square
  weight * (y_true - np.

Mean Squared Error: 64.46044158935547
R-squared: 1.0


# n_partions = 200

In [14]:
import time
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask.diagnostics import ProgressBar

# XGBoost related imports
from xgboost.dask import DaskXGBRegressor

# Metrics for model evaluation
from sklearn.metrics import mean_squared_error, r2_score

# Initialize Dask Client
client = Client()  # You can specify a scheduler address or leave it for local cluster

# Your dataset needs to be loaded as Dask DataFrames (assuming X_train, y_train, X_test, y_test are Dask DataFrames)
# Example:
# X_train = dd.from_pandas(pandas_X_train, npartitions=4)
# y_train = dd.from_pandas(pandas_y_train, npartitions=4)
# X_test = dd.from_pandas(pandas_X_test, npartitions=4)
# y_test = dd.from_pandas(pandas_y_test, npartitions=4)

# Initialize the Dask-XGBoost Regressor
xgb_model = DaskXGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=3)

# Train the Model
print("Training the model...")
start_time = time.time()  # Start timer

with ProgressBar():
    xgb_model.fit(X_train, y_train)
end_time = time.time()  # End timer

# Calculate training time
training_time = end_time - start_time
print(f"Training completed in {training_time:.2f} seconds.")

# Make Predictions
print("Predicting...")
y_pred = xgb_model.predict(X_test)

# Evaluate the Model
y_test = y_test.compute()  # Bring to memory
y_pred = y_pred.compute()  # Bring to memory

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Shut down the client
client.close()


Windows is not officially supported for dask/xgboost, contribution are welcomed.
Windows is not officially supported for dask/xgboost, contribution are welcomed.


Training the model...


c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.55 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 269.64 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Windows is not officially supported for dask/xgboost, contribution are welcomed.
Windows is not officially supported for dask/xgboost, contribution are welcomed.


Training completed in 80.68 seconds.
Predicting...


c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 269.64 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\distributed\client.py:3164: UserWarning: Sending large graph of size 862.56 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:522: RuntimeWarning: overflow encountered in cast
  scl = avg_as_array.dtype.type(a.size/avg_as_array.size)
c:\Users\Himanshu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1198: RuntimeWarning: overflow encountered in square
  weight * (y_true - np.average(y_true, axis=0, weights=sample_weight)) ** 2


Mean Squared Error: 66.9477767944336
R-squared: 1.0
